# Manipulating data - Examples in Pandas

In [1]:
# imports

import datetime as DT
import numpy as N
import pandas as P

#### Basic operations

Operations on `Series` and `DataFrame` objects are typically performed in an element-wise manner.

In [2]:
# Series object
s = P.Series([2, 1.5, 9, 4])

In [3]:
s

0    2.0
1    1.5
2    9.0
3    4.0
dtype: float64

In [4]:
# adding 1 to all elements
s + 1

0     3.0
1     2.5
2    10.0
3     5.0
dtype: float64

In [5]:
# adding together two Series objects with identical dimensions, element by element
s + s

0     4.0
1     3.0
2    18.0
3     8.0
dtype: float64

In [6]:
# calculating the squares of all elements
s**2

0     4.00
1     2.25
2    81.00
3    16.00
dtype: float64

In [7]:
# calculating the square roots of all elements with a NumPy function
N.sqrt(s)

0    1.414214
1    1.224745
2    3.000000
3    2.000000
dtype: float64

#### CAUTION

Pandas matches elements based on their names, whereas NumPy uses position-based matching. Consider using `reset_index()` in Pandas, if necessary, to perform position-based operations.

In [8]:
# Series objects with non-matching row names
s1 = P.Series([2, 1.5, 9, 4])
s2 = P.Series([2, 1.5, 9, 4],
              index=[0, 1, 2, 4])

In [9]:
s1

0    2.0
1    1.5
2    9.0
3    4.0
dtype: float64

In [10]:
s2

0    2.0
1    1.5
2    9.0
4    4.0
dtype: float64

In [11]:
# adding together elements matched by their names
# (NaN values are produced for non-matched elements)
s1 + s2

0     4.0
1     3.0
2    18.0
3     NaN
4     NaN
dtype: float64

In [12]:
# adding together elements matched by their position (as NumPy arrays)
s1.values + s2.values

array([ 4.,  3., 18.,  8.])

In [13]:
# using reset_index() to replace current row names with consecutive integers
# before adding Series objects together
s1.reset_index(drop=True) + s2.reset_index(drop=True)

0     4.0
1     3.0
2    18.0
3     8.0
dtype: float64

In [14]:
# DataFrame objects with non-matching row and column names
df1 = P.DataFrame([range(1, 4), range(11, 14)],
                  index=['a', 'c'],
                  columns=['x', 'y', 'z'])
df2 = P.DataFrame(zip(range(2, 5), range(12, 15)),
                  index=['a', 'b', 'c'],
                  columns=['x', 'z'])

In [15]:
df1

,x,y,z
a,1,2,3
c,11,12,13


In [16]:
df2

,x,z
a,2,12
b,3,13
c,4,14


In [17]:
# calculating element-wise products, matching rows and columns by name
# (only some row-column combinations exist in both DataFrame objects)
df1 * df2

,x,y,z
a,2.0,NaN,36.0
b,NaN,NaN,NaN
c,44.0,NaN,182.0


Some functions reduce the elements into a single (or several) values.

In [18]:
# calculating the sum of all elements 
s.sum()

16.5

In [19]:
# as above but with a NumPy function
N.sum(s)

16.5

In [20]:
# calculating some basic statistics
# (multiple aggregated values are produced simultaneously)
s.describe()

count    4.000000
mean     4.125000
std      3.424787
min      1.500000
25%      1.875000
50%      3.000000
75%      5.250000
max      9.000000
dtype: float64

Complex calculations can be expressed in a straightforward and readable manner.

In [21]:
# Z-standardisation
# (for each element, subtract the mean and divide by the standard deviation)
(s - s.mean()) / s.std()

0   -0.620477
1   -0.766471
2    1.423446
3   -0.036499
dtype: float64

Some operations on `DataFrame` objects are performed in a row-wise or column-wise manner.

In [22]:
# DataFrame object
df = P.DataFrame(zip(range(12, 16),
                     ['low', 'low', 'high', 'low'],
                     [2.3, 0.7, 1.1, 0.2]),
                 index=['a', 'b', 'c', 'd'],
                 columns=['x', 'y', 'z'])

In [23]:
df

,x,y,z
a,12,low,2.3
b,13,low,0.7
c,14,high,1.1
d,15,low,0.2


In [24]:
# calculating some basic statistics for each column
# (categorical variables are treated differently from quantitative variables)
df.describe(include='all')

,x,y,z
count,4.000000,4,4.000000
unique,NaN,2,NaN
top,NaN,low,NaN
freq,NaN,3,NaN
mean,13.500000,NaN,1.075000
std,1.290994,NaN,0.895824
min,12.000000,NaN,0.200000
25%,12.750000,NaN,0.575000
50%,13.500000,NaN,0.900000
75%,14.250000,NaN,1.400000


In [25]:
# calculating sums over columns (dimension 1)
df[['x', 'z']].sum(axis=1)

a    14.3
b    13.7
c    15.1
d    15.2
dtype: float64

In [26]:
# calculating sums over rows (dimension 0)
df[['x', 'z']].sum(axis=0)

x    54.0
z     4.3
dtype: float64

In [27]:
# subtracting the corresponding column mean from each element
# (the means are broadcast along the row dimension such that
#  the same number is used for each element in a column;
#  'y' column contains NaNs because columns are matched by name
#  and there is no mean for 'y')
df - df[['x', 'z']].mean(axis=0)

,x,y,z
a,-1.5,NaN,1.225
b,-0.5,NaN,-0.375
c,0.5,NaN,0.025
d,1.5,NaN,-0.875


#### Manipulating data

With the use of data selection operations, multiple simultaneous assignments can be performed directly to a Pandas object. There are also convenience functions for commonly encountered situations. Some modifications can be made either in-place or by creating a new Pandas object. 

In [28]:
# DataFrame to be modified
df = P.DataFrame([[DT.date(2011,10,4), 'river', 15.4, 0.01],
                  [DT.date(2017,3,14), 'river', 6.6, 0.03],
                  [DT.date(2014,5,17), 'river', 5.9, 0.03],
                  [DT.date(2015,4,2), 'river', 6.2, 0.04],
                  [DT.date(1970,1,1), 'lake', 6.8, 0.01],
                  [DT.date(2022,1,5), 'lake', 9.5, 0.02],
                  [DT.date(2022,6,9), 'lake', 7.9, 0.01],
                  [DT.date(2029,3,21), 'sea', 5.8, 3.1],
                  [DT.date(2021,2,2), 'sea', 6.7, 3.4],
                  [DT.date(2020,8,18), 'sea', 6.1, N.nan]],
                 columns=['date', 'body_of_water', 'pH', 'salinity'])

In [29]:
df

,date,body_of_water,pH,salinity
0,2011-10-04,river,15.4,0.01
1,2017-03-14,river,6.6,0.03
2,2014-05-17,river,5.9,0.03
3,2015-04-02,river,6.2,0.04
4,1970-01-01,lake,6.8,0.01
5,2022-01-05,lake,9.5,0.02
6,2022-06-09,lake,7.9,0.01
7,2029-03-21,sea,5.8,3.10
8,2021-02-02,sea,6.7,3.40
9,2020-08-18,sea,6.1,NaN


In [30]:
# replacing invalid (too high) pH values with NaN
df.loc[df.pH > 10, 'pH'] = N.nan

In [32]:
# replacing invalid dates with NaN
# (1970-01-01 is likely a placeholder for a missing value;
#  the dates in the future are also invalid)
df.loc[(df.date == DT.date(1970,1,1)) | (df.date > DT.date(2022,10,1)), 'date'] = N.nan

In [33]:
df

,date,body_of_water,pH,salinity
0,2011-10-04,river,NaN,0.01
1,2017-03-14,river,6.6,0.03
2,2014-05-17,river,5.9,0.03
3,2015-04-02,river,6.2,0.04
4,NaN,lake,6.8,0.01
5,2022-01-05,lake,9.5,0.02
6,2022-06-09,lake,7.9,0.01
7,NaN,sea,5.8,3.10
8,2021-02-02,sea,6.7,3.40
9,2020-08-18,sea,6.1,NaN


#### CAUTION

Using chained selections will not modify the original `DataFrame` object because a copy is created between the selections (and the copy is modified).

In [34]:
# DataFrame object
bad = P.DataFrame(zip(range(4), [0, 0, 0, 0]),
                  index=['a', 'b', 'c', 'd'],
                  columns=['x', 'y'])

In [ ]:
bad

In [35]:
# trying set 'y' value to 10 if 'x' value is greater than 1
bad[bad.x > 1].y = 10

/tmp/ipykernel_29508/2839995548.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bad[bad.x > 1].y = 10


In [36]:
# (failed because modifications were made to "a copy of a slice from a DataFrame")
bad

,x,y
a,0,0
b,1,0
c,2,0
d,3,0


In [37]:
# selecting the desired rows and the target column in a single step
bad.loc[bad.x > 1, 'y'] = 10

In [38]:
bad

,x,y
a,0,0
b,1,0
c,2,10
d,3,10


There are convenience functions to handle missing values.

In [ ]:
# finding missing values in an element-wise manner
df.isna()

In [ ]:
# replacing all missing values with a constant
# (this creates a new object)
df.fillna(0)

In [39]:
# replacing missing pH and salinity values with the corresponding column median
# (dates will not be modified because no median is provided for the 'date' column)
df.fillna(df[['pH', 'salinity']].median())

,date,body_of_water,pH,salinity
0,2011-10-04,river,6.6,0.01
1,2017-03-14,river,6.6,0.03
2,2014-05-17,river,5.9,0.03
3,2015-04-02,river,6.2,0.04
4,NaN,lake,6.8,0.01
5,2022-01-05,lake,9.5,0.02
6,2022-06-09,lake,7.9,0.01
7,NaN,sea,5.8,3.10
8,2021-02-02,sea,6.7,3.40
9,2020-08-18,sea,6.1,0.03


In [41]:
# as above but replacing values in-place
df.fillna(df[['pH', 'salinity']].median(), inplace=True)

In [42]:
df

,date,body_of_water,pH,salinity
0,2011-10-04,river,6.6,0.01
1,2017-03-14,river,6.6,0.03
2,2014-05-17,river,5.9,0.03
3,2015-04-02,river,6.2,0.04
4,NaN,lake,6.8,0.01
5,2022-01-05,lake,9.5,0.02
6,2022-06-09,lake,7.9,0.01
7,NaN,sea,5.8,3.10
8,2021-02-02,sea,6.7,3.40
9,2020-08-18,sea,6.1,0.03


In [ ]:
# replacing missing dates with Jan 1, 1970 (which should not be done in a real analysis)
# (modifying a single DataFrame column in-place, without creating a copy)
df.date.fillna(DT.date(1970,1,1), inplace=True)

In [ ]:
df

In [ ]:
# finding rows in which the value of 'body_of_water' is in the given list
idx = df.body_of_water.isin(['river', 'lake'])
# combining 'river' and 'lake' values into a single 'fresh' value
df.loc[idx, 'body_of_water'] = 'fresh'

In [ ]:
df

Rows can be sorted either by row names or by values in one or more columns.

In [ ]:
# moving the 'date' column to be the row names
df.set_index('date', inplace=True)

In [ ]:
df

In [ ]:
# sorting by row names
df.sort_index()

In [ ]:
# sorting by pH values, descending order
df.sort_values('pH', ascending=False)

In [ ]:
# sorting primarily by salinity and secondarily by pH
df.sort_values(['salinity', 'pH'])

#### Split-apply-combine paradigm

Pandas provides `groupby()` to specify groups. Aggregation can be achieved with `aggregate()` or by using one of the built-in aggregation functions. There is also `transform()` for transformations, `filter()` for filtering, and `apply()` for custom calculations.

In [ ]:
# DataFrame to be analysed
df = P.DataFrame([[DT.date(2011,10,4), 'river', 7.4, 0.01],
                  [DT.date(2017,3,14), 'river', 6.6, 0.03],
                  [DT.date(2014,5,17), 'river', 5.9, 0.03],
                  [DT.date(2015,4,2), 'river', 6.2, 0.04],
                  [DT.date(2017,1,2), 'lake', 6.8, 0.01],
                  [DT.date(2022,1,5), 'lake', 9.5, 0.02],
                  [DT.date(2022,6,9), 'lake', 7.9, 0.01],
                  [DT.date(2019,9,9), 'sea', 5.8, 3.1],
                  [DT.date(2021,2,2), 'sea', 6.7, 3.4],
                  [DT.date(2020,8,18), 'sea', 6.1, 2.8]],
                 columns=['date', 'body_of_water', 'pH', 'salinity'])

In [ ]:
df

In [ ]:
# grouping rows by 'body_of_water'
grouped = df.groupby('body_of_water')

In [ ]:
# accessing groups one by one
for name, group in grouped:
    print(name, end="\n\n")
    print(group, end="\n\n")

In [ ]:
# calculating group-wise means and medians of pH and salinity
grouped[['pH', 'salinity']].aggregate([N.mean, N.median])

In [ ]:
# like above but with the built-in Pandas function for mean
# (the 'date' column is automatically ignored here)
grouped.mean()

In [ ]:
# function to subtract the corresponding column mean from each element (given a dataframe)
fn = lambda df: df - df.mean()
# transforming values by subtracting within-group means
transformed = grouped[['pH', 'salinity']].transform(fn)
# restoring the 'date' and 'body_of_water' columns
P.concat([df[['date', 'body_of_water']], transformed], axis=1)

In [ ]:
# function to return a Boolean value that describes the given DataFrame
fn = lambda df: df.salinity.mean() < 1
# selecting rows that belong to a group in which the mean 'salinity' is below one
grouped.filter(fn)

In [ ]:
# function to return a modified DataFrame
fn = lambda df: df[df.salinity < df.salinity.mean()]
# selecting rows in which 'salinity' is below the group mean
grouped.apply(fn)

In [ ]:
# function to calculate the number of acidic and alkaline observations
# as well as the pH and salinity of the observation with the extreme pH
def fn(df):
    n_acid = (df.pH < 7).sum()
    n_base = (df.pH > 7).sum()
    return P.DataFrame([[n_acid,
                         N.nan if n_acid == 0 else df.iloc[df.pH.argmin()].pH,
                         N.nan if n_acid == 0 else df.iloc[df.pH.argmin()].salinity],
                        [n_base,
                         N.nan if n_base == 0 else df.iloc[df.pH.argmax()].pH,
                         N.nan if n_base == 0 else df.iloc[df.pH.argmax()].salinity]],
                       index=['acidic', 'alkaline'],
                       columns=['count', 'pH_of_extreme', 'salinity_of_extreme'])
# applying the function to each group
grouped.apply(fn)

In [ ]:
# 'acidic vs. alkaline' status
# (naming a Boolean vector properly and replacing its values with human-readable strings)
status = (df.pH < 7).rename('status').map({True: 'acidic', False: 'alkaline'})
# using the Boolean vector as a secondary grouping criterion
df.groupby(['body_of_water', status]).mean()